In [15]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
llm = ChatOpenAI(
    api_key = os.getenv("OPENROUTER_API_KEY"),
    base_url = "https://openrouter.ai/api/v1",
    model = "google/gemini-2.5-flash",
    max_completion_tokens=200

)

In [17]:
INTENT_PROMPT = """You are an IntentParser for a restaurant assistant.

Your job is to:
1. Classify the user's message into one or more allowed intents.
2. Extract the slots defined for each intent.

Allowed intents:
- menu — user asks about items, availability, filtering (veg/non-veg/price/type), or search.
- knowledge_base — user asks about restaurant policies, FAQs, or other general information.
- chit_chat — casual conversation or greetings.
- human_escalation — tasks that require human intervention (e.g., placing an order, complaints).
- ambiguous — cannot confidently classify.

Output format (for storing in state):
- Produce a list of tasks, where each task contains:
  - intent: one of the allowed intents
  - slots: dictionary with relevant keys (see below)
  - tool_name: string indicating which tool to call for this intent (menu_tool, kb_tool, or None)
  - result: null (to be filled later by tool)

Slot schemas:
- menu:
    - search: string | null
    - type: "veg" | "nonveg" | null
    - price_min: number | null
    - price_max: number | null
- knowledge_base:
    - query: string | null
- chit_chat:
    - slots: empty dictionary {}
- human_escalation:
    - slots: empty dictionary {}
- ambiguous:
    - slots: empty dictionary {}

Rules:
1. For menu intent, always return the same keys (`search`, `type`, `price_min`, `price_max`). Use null if missing.
2. For knowledge_base, extract the part of the user query relevant to retrieving data as `query`.
3. If multiple distinct intents are present in the user query, split them into separate tasks.
4. Include `tool_name` as `"menu_tool"` for menu, `"kb_tool"` for knowledge_base, and `None` for chit_chat, human_escalation, or ambiguous.
5. Always set `result` to null (tools will fill it later).
6. Return **only the list of tasks**; do not generate any final answer.

Few-shot examples:

User: "Do you have veg pizzas under 12?"
Tasks:
[
  {
    "intent": "menu",
    "slots": {"search":"pizza","type":"veg","price_min":null,"price_max":12},
    "tool_name": "menu_tool",
    "result": null
  }
]

User: "What time do you close on Fridays?"
Tasks:
[
  {
    "intent": "knowledge_base",
    "slots": {"query":"closing hours on Fridays"},
    "tool_name": "kb_tool",
    "result": null
  }
]

User: "Hi, how are you?"
Tasks:
[
  {
    "intent": "chit_chat",
    "slots": {},
    "tool_name": null,
    "result": null
  }
]

User: "I want to place an order for two burgers."
Tasks:
[
  {
    "intent": "human_escalation",
    "slots": {},
    "tool_name": null,
    "result": null
  }
]

User: "Tell me if you have paneer pizza and your delivery areas."
Tasks:
[
  {
    "intent": "menu",
    "slots": {"search":"pizza","type":"veg","price_min":null,"price_max":null},
    "tool_name": "menu_tool",
    "result": null
  },
  {
    "intent": "knowledge_base",
    "slots": {"query":"delivery areas"},
    "tool_name": "kb_tool",
    "result": null
  }
]

Final instruction:
Respond only with the JSON array of tasks as described above. Do not include commentary or extra text.

"""

In [18]:
from typing import TypedDict, List, Dict, Any
from langchain.schema import BaseMessage

class TaskDict(TypedDict):
    intent: str                 # "menu", "knowledge_base", "chitchat", etc.
    confidence: float           # model confidence
    slots: Dict[str, Any]       # plain dict of slots
    result: Any                 # placeholder for tool output (can be None)

class State(TypedDict):
    messages: List[BaseMessage]   # conversation history
    tasks: List[TaskDict]         # list of classified tasks

In [19]:
from pydantic import BaseModel
from typing import Literal
class Task(BaseModel):
    intent: Literal["menu", "knowledge_base", "chitchat", "human_escalation", "ambiguous"]
    confidence: float
    slots: Dict[str, Any] = {}

class IntentOutput(BaseModel):
    tasks: List[Task]

In [20]:
from langchain.schema import SystemMessage, BaseMessage
def intent_parser(state: Dict[str, Any], INTENT_PROMPT: str, llm) -> Dict[str, Any]:

    # prepare messages (SystemMessage + conversation)
    messages: List[BaseMessage] = [SystemMessage(content=INTENT_PROMPT)] + state.get("messages", [])

    # get a structured LLM that returns IntentOutput
    llm_structured = llm.with_structured_output(IntentOutput)

    # invoke model (it returns a parsed IntentOutput instance)
    resp = llm_structured.invoke(messages)  # resp is a Pydantic-like object matching IntentOutput

    normalized_tasks = []
    for t in resp.tasks:
        slots = t.slots or {}
        normalized_tasks.append({
            "intent": t.intent,
            "confidence": float(t.confidence or 0.0),
            "slots": slots,
            "result": None
        })

    # Return minimal state fragment
    return {
        "messages": [resp],      # model message
        "tasks": normalized_tasks
    }

In [33]:
q = "do you guys do anything spicy but it has to be veg and also what time do you guys open"
from langchain.schema import HumanMessage
state: State = {"messages": [HumanMessage(content=q)],
                "intent": None, "confidence": None, "slots": {}}
state = intent_parser(state,INTENT_PROMPT, llm)

In [34]:
state["tasks"][0]["slots"]

{'search': 'spicy', 'type': 'veg', 'price_min': None, 'price_max': None}

In [ ]:
for task in state["tasks"]:
    if task["intent"] == "menu":
        task["result"] = menu_tool(task["slots"])
    elif task["intent"] == "knowledge_base":
        task["result"] = kb_tool(task["slots"])

In [26]:
from urllib.parse import urlencode, quote_plus
def build_menu_uri(base_url: str, s: Dict[str, Any]) -> str:
    """
    Example: base_url = "https://api.example.com/menu/search"
    Produces: https://api.example.com/menu/search?search=pizza&type=veg&price_max=12
    """
   
    params = {}
    if s["search"]:
        params["search"] = s["search"]
    if s["type"]:
        params["type"] = s["type"]
    if s["price_min"] is not None:
        params["price_min"] = str(s["price_min"])
    if s["price_max"] is not None:
        params["price_max"] = str(s["price_max"])
    if params:
        return f"{base_url}?{urlencode(params, quote_via=quote_plus)}"
    return base_url

In [35]:
uri = build_menu_uri("https://api.example.com/menu/search",state["tasks"][0]["slots"])

In [36]:
uri

'https://api.example.com/menu/search?search=spicy&type=veg'